# Four Specialties, Multiple Locations  
- Read a CSV file containing specialties, locations and physicians

In [1]:
import os
import openai
import webbrowser
import datetime 
import json
from json import loads, dumps
import csv
import pandas as pd
import subprocess

# from CSV_to_JSON_or_DICT import CSV_to_DICT




from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import write_df_to_DOCX
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import pp_json
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import get_completion_from_messages
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage
from chatGPT_utilities import openai_object_to_dict
from chatGPT_utilities import CSV_to_DICT

openai.api_key = setup_OpenAI_API()


ImportError: cannot import name 'write_df_to_DOCX' from 'chatGPT_utilities' (/Users/michael/Coding/Clinic Availability/chatGPT_utilities.py)

In [2]:
url_chatGPT_usage = 'https://platform.openai.com/account/usage'
webbrowser.open_new(url_chatGPT_usage)

True

## Read Data
- CSV file
- Convert to dict --> JSON

### Read Clinic Locations and Phone Numbers

In [2]:
# def CSV_to_DICT(file_name):
#     # csv_file = "4_specialties_multiple_locations.csv"  # Replace with the path to your CSV file
#     assert '.csv' in file_name, f'{csv_file} is NOT a CSV file'
#     data = {}

#     # Read the CSV file
#     with open(file_name, mode="r") as file:
#         reader = csv.reader(file)
#         headers = next(reader)  # Read the header row

#         for row in reader:
#             primary_key = row[0]  # Primary key from the first column
#             secondary_key = row[1]  # Secondary key from the second column
#             values = row[2:]  # Values from the remaining columns

#             if primary_key in data:
#                 # Primary key already exists, make the secondary key a nested dictionary
#                 if secondary_key not in data[primary_key]:
#                     data[primary_key][secondary_key] = values
#             else:
#                 # Primary key doesn't exist, create a new entry
#                 data[primary_key] = {secondary_key: values}

#     # Print the resulting dictionary
#     # print(data)
#     return data


In [22]:
csv_file_path  = '4_specialties_multiple_locations_phone.csv'
csv_file_path  = 'physicians_2023-06-18.csv'
# csv_file_path  = 'ortho_multiple_locations.csv'

df = pd.read_csv(csv_file_path)
df
print(df.columns)
df = df.sort_values(['Specialty'], ascending=[False])
df = df.reset_index(drop=True)



Index(['Physician', 'Specialty', 'Clinic', 'Location', 'Phone', 'Clinic_Days',
       'Rating', 'Education', 'Years_in_Practice'],
      dtype='object')


In [23]:
import pandas as pd
from tabulate import tabulate

# Assuming you have a DataFrame named "df"
# ... code to create or obtain the DataFrame ...

# Convert DataFrame to Markdown format
markdown_table = tabulate(df, headers='keys', tablefmt='pipe')

# Print or display the Markdown table
print(markdown_table)



|    | Physician    | Specialty         | Clinic   | Location             | Phone        | Clinic_Days                          |   Rating | Education                                 |   Years_in_Practice |
|---:|:-------------|:------------------|:---------|:---------------------|:-------------|:-------------------------------------|---------:|:------------------------------------------|--------------------:|
|  0 | Physician_5  | Pediatrics        | Clinic_A | 123 Pine St, Anytown | 111-111-3890 | ['Thursday', 'Tuesday', 'Monday']    |      4.6 | {'Med_school': 'UCDavis'}                 |                  21 |
|  1 | Physician_11 | Pediatrics        | Clinic_C | 123 Pine St, Anytown | 111-111-4558 | ['Tuesday', 'Friday', 'Thursday']    |      4.8 | {'Med_school': 'UCDavis'}                 |                  16 |
|  2 | Physician_1  | Orthopedics       | Clinic_B | 123 Pine St, Anytown | 111-111-7911 | ['Monday', 'Thursday', 'Tuesday']    |      4.4 | {'Med_school': 'University of W

# Data  

|    | Physician    | Specialty         | Clinic   | Location             | Phone        | Clinic_Days                          |   Rating | Education                                 |   Years_in_Practice |
|---:|:-------------|:------------------|:---------|:---------------------|:-------------|:-------------------------------------|---------:|:------------------------------------------|--------------------:|
|  0 | Physician_5  | Pediatrics        | Clinic_A | 123 Pine St, Anytown | 111-111-3890 | ['Thursday', 'Tuesday', 'Monday']    |      4.6 | {'Med_school': 'UCDavis'}                 |                  21 |
|  1 | Physician_11 | Pediatrics        | Clinic_C | 123 Pine St, Anytown | 111-111-4558 | ['Tuesday', 'Friday', 'Thursday']    |      4.8 | {'Med_school': 'UCDavis'}                 |                  16 |
|  2 | Physician_1  | Orthopedics       | Clinic_B | 123 Pine St, Anytown | 111-111-7911 | ['Monday', 'Thursday', 'Tuesday']    |      4.4 | {'Med_school': 'University of Wisconsin'} |                  14 |
|  3 | Physician_2  | Orthopedics       | Clinic_B | 123 Pine St, Anytown | 111-111-1098 | ['Wednesday', 'Tuesday', 'Thursday'] |      3.1 | {'Med_school': 'University of Miami'}     |                  25 |
|  4 | Physician_4  | Orthopedics       | Clinic_B | 123 Pine St, Anytown | 111-111-6200 | ['Wednesday', 'Tuesday', 'Monday']   |      4.9 | {'Med_school': 'University of Wisconsin'} |                  12 |
|  5 | Physician_10 | Orthopedics       | Clinic_B | 123 Pine St, Anytown | 111-111-7448 | ['Thursday', 'Monday', 'Wednesday']  |      3.2 | {'Med_school': 'University of Michigan'}  |                  14 |
|  6 | Physician_12 | Orthopedics       | Clinic_A | 123 Pine St, Anytown | 111-111-5186 | ['Thursday', 'Monday', 'Friday']     |      5   | {'Med_school': 'UCDavis'}                 |                   6 |
|  7 | Physician_9  | <span style="color:red">Oncology</span>          | Clinic_C | 123 Pine St, Anytown | 111-111-3164 | ['Tuesday', 'Wednesday', 'Thursday'] |      3.2 | {'Med_school': 'University of Miami'}     |                  12 |
|  8 | Physician_13 | <span style="color:red">Oncology</span>          | Clinic_C | 123 Pine St, Anytown | 111-111-1620 | ['Friday', 'Tuesday', 'Wednesday']   |      3.9 | {'Med_school': 'University of Michigan'}  |                   6 |
|  9 | Physician_15 | <span style="color:red">Oncology</span>          | Clinic_C | 123 Pine St, Anytown | 111-111-9124 | ['Friday', 'Wednesday', 'Tuesday']   |      3.6 | {'Med_school': 'University of Wisconsin'} |                  11 |
| 10 | Physician_7  | OBGYN             | Clinic_A | 123 Pine St, Anytown | 111-111-5842 | ['Monday', 'Thursday', 'Wednesday']  |      3   | {'Med_school': 'University of Miami'}     |                   9 |
| 11 | Physician_8  | OBGYN             | Clinic_B | 123 Pine St, Anytown | 111-111-5879 | ['Wednesday', 'Thursday', 'Tuesday'] |      4.8 | {'Med_school': 'UCDavis'}                 |                  21 |
| 12 | Physician_6  | Internal Medicine | Clinic_C | 123 Pine St, Anytown | 111-111-9954 | ['Friday', 'Wednesday', 'Monday']    |      3   | {'Med_school': 'University of Michigan'}  |                  22 |
| 13 | Physician_14 | Internal Medicine | Clinic_C | 123 Pine St, Anytown | 111-111-8926 | ['Tuesday', 'Friday', 'Thursday']    |      4.3 | {'Med_school': 'University of Miami'}     |                  21 |
| 14 | Physician_3  | Cardiology        | Clinic_C | 123 Pine St, Anytown | 111-111-8963 | ['Wednesday', 'Tuesday', 'Thursday'] |      4.3 | {'Med_school': 'University of Wisconsin'} |                  23 |

<span style="color:blue">Oncology</span>


 

file_path = 'dataframe.docx'
write_df_to_DOCX(file_path)
_ = subprocess.call(['open',  file_path])

## Read CSV To List Of Dictionaries

In [15]:
import csv
import ast



    
    # def sort_clinic_days():    
    #     for ele in data_list:
    #         for k,value in ele.items():
    #             if k == 'clinic_days':
    #                 print(type(value))
    #                 print( ast.literal_eval(value))
    #                 list_of_clinic_days = ast.literal_eval(value)
    #                 print(list_of_clinic_days)
    #                 sorted_weekdays = sorted(list_of_clinic_days, 
    #                                          key=lambda day: ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'].index(day))
    #                 ele[k] = sorted_weekdays
    #                 print((ele[k]))
    #         print('\n'*2 )
    # return data_list

def read_csv_to_list_of_dictionaries(filename):  
    """
    Description:
        This function reads a CSV file and converts its contents into a list of dictionaries. 
        Each dictionary represents a row in the CSV file, 
        where the keys are the column names and the values are the corresponding values from the row.
    Parameters:
        filename (str): The path and name of the CSV file to read.
    Returns:
        result_list (list): A list of dictionaries, where each dictionary represents a row from the CSV file.
    """

    
    result_list = []

    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            dict_small = dict(row)
            
            dict_small['Clinic_Days'] = ast.literal_eval(dict_small['Clinic_Days'])
            
            sorted_weekdays = sorted(dict_small['Clinic_Days'] , 
                                             key=lambda day: ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'].index(day))
            dict_small['Clinic_Days'] = sorted_weekdays
            result_list.append(dict_small)
       
    return result_list



def is_weekday_in_order(weekdays):
    """
        Description:
        This function checks if a given list of weekdays is in ascending order,
        according to the conventional order of days in a week (Sunday, Monday, Tuesday, etc.). 
        It returns True if the weekdays are in order and False otherwise.

        Parameters:
            weekdays (list): A list of strings representing weekdays.
        Returns:
            is_in_order (bool): True if the weekdays are in order, False otherwise.
    # Example usage
    # weekdays = ['Tuesday', 'Wednesday', 'Thursday']
    # assert is_weekday_in_order(weekdays), The weekdays are not in the normal order.    """

    
    normal_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    weekdays_indices = [normal_order.index(day) for day in weekdays]
    return weekdays_indices == sorted(weekdays_indices)


In [16]:
# Example usage
filename  = 'physicians_2023-06-18.csv'
data_list = read_csv_to_list_of_dictionaries(filename)
print(filename + ' has been read')
for doctor in data_list:
    assert is_weekday_in_order(doctor['Clinic_Days']), "The weekdays are not in the normal order for " \
    ' Doctor ' + doctor['Physician']
print('All clinic days are in proper order')

physicians_2023-06-18.csv has been read
All clinic days are in proper order


Some Markdown text with <span style="color:red">some *red* text</span>.

<div class="alert alert-block alert-danger">
What is a doctor is at 2 different sites?
</div>

 

In [23]:
# specialty_dict = {}
# for physician, info in data.items():
#     specialty = list(info.keys())[0]
#     if specialty not in specialty_dict:
#         specialty_dict[specialty] = []
#     specialty_dict[specialty].append({physician: info[specialty]})

# pp_json(specialty_dict)

In [24]:
# pp_json(json_clinics)
# clinics_dict

### Read Doctors' Clinic Locations and Availability

In [25]:
def create_dictionary_doctors_locations(csv_file):
    dictionary = {}
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row if it exists
        sorted_data = sorted(reader, key=lambda row: row[0])  # Sort the data by clinic name
        for row in sorted_data:
            clinic_name = row[0]
            doctor_name = row[1]
            availability = row[2]
            if clinic_name not in dictionary:
                dictionary[clinic_name] = {}
            dictionary[clinic_name][doctor_name] = availability
    return dictionary

# Usage example
file_doctors_locations = 'Doctors_at_Clinic_Locations.csv'
doctors_dict = create_dictionary_doctors_locations(file_doctors_locations)
pp_json(doctors_dict)

# json_doctors_locations_avail = json.dumps(doctors_dict)
# print(f'data is of {type(json_doctors_locations_avail)}')
# pp_json(json_doctors_locations_avail)
print(f'We are using "clinics_dict" and "doctors_dict"')

We are using "clinics_dict" and "doctors_dict"


In [ ]:
# May want to use this in the future


# Output a python list of objects, where each object has \
# the following format:
#     'category': <one of Computers and Laptops, \
#     Smartphones and Accessories, \
#     Televisions and Home Theater Systems, \
#     Gaming Consoles and Accessories, 
#     Audio Equipment, Cameras and Camcorders>,
# OR
#     'products': <a list of products that must \
#     be found in the allowed products below>

# System Message  

<div class="alert alert-block alert-danger">
    Interesting that when the clinics are in JSON format, it hallucinates in having an  ophthalmology clinic, but it does not do that when using a dictionary.

</div>

`json_clinics` is JSON format STR

In [23]:
# clinics_dict
The following lists the doctors, where they work and what days of the week they are working:
{doctors_dict}



In [8]:
mask = df['Specialty'] == 'Orthopedics'
df[mask] 

,Physician,Specialty,Clinic,Location,Phone,Clinic_Days,Rating,Education,Years_in_Practice
0,Physician_1,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7911,"['Monday', 'Thursday', 'Tuesday']",4.4,{'Med_school': 'University of Wisconsin'},14
1,Physician_2,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-1098,"['Wednesday', 'Tuesday', 'Thursday']",3.1,{'Med_school': 'University of Miami'},25
3,Physician_4,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-6200,"['Wednesday', 'Tuesday', 'Monday']",4.9,{'Med_school': 'University of Wisconsin'},12
9,Physician_10,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7448,"['Thursday', 'Monday', 'Wednesday']",3.2,{'Med_school': 'University of Michigan'},14
11,Physician_12,Orthopedics,Clinic_A,"123 Pine St, Anytown",111-111-5186,"['Thursday', 'Monday', 'Friday']",5.0,{'Med_school': 'UCDavis'},6


In [28]:
delimiter = "####"
system_message = f"""
Act as if you are a customer service rep. \
The patient's query will be delimited with \
{delimiter} characters.

The following lists are Clinics: 
{data_list}

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} Decide what specialty would be appropriate 

Step 3:{delimiter} Find all clinics  that have the  correct specialty

Step 4:{delimiter} Determine which doctors are working at the  
appropriate specialty clinic on which days


Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 2 reasoning>
Step 4:{delimiter} <step 2 reasoning>

Be sure to check all clinics to see if they an appropriate specialist for the medical problem 

If we do not have a medical clinic that is relevant to the patient's problem
, respond in a polite manner and advise them to search the web at www.google.com, using the URL 
link .



If the patient has a question that could be managed by a medical clinic
that is on the list, then list each of the relevant clinics in the following format:
Clinic      :
Location    :
Phone       :
Doctors     : 

For each doctor in the clinic, on a separate line and indented 10 spaces 
list the Doctors's name then a colon (:) and  which days of the week they work at the clinic

On a separate line, inform the patient which days the clinic is availble.
Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.

Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.

Answer the customer in a friendly tone. Express concern that they are having a problem.
"""

# Broken finger  
data_list

{clinics_dict}


In [12]:
delimiter = "####"
system_message = f"""
Acting as a customer service representative, respond to the patient \
The patient's query will be delimited with \
{delimiter} characters.

The following lists are Clinics: 
{data_list}

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} Decide what specialty would be appropriate 

Step 3:{delimiter} Create a python list of the set of all doctors in that specialty

Step 4:{delimiter} Create a python list of the set of all clinics in that specialty


Step 5:{delimiter} Create a python dictionary 
where the primary key is the medical specialty
and the secondary keys are the names of the clinic in that speciality
the tertiary keys are the names of the doctor in that specialty and 
the value is a list which are the days of the week that the doctor works in the clinic

Step 6:{delimiter} Determine if the patient has any restrictions on the day of the week.
If there are restrictions, then remove those doctors that are not working on the day of the week the 
patient specifed



Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 5 reasoning>
Step 6:{delimiter} <step 6 reasoning>



If there are apprpriate clinics for the patient h
then list each of the relevant clinics in the following format:
Clinic      :
Doctor # 1     : 
    Phone Number :  
    Days         : clinic days sorted with Monday as first day of week and Friday as last day of the week
Doctor # 2     
    Phone Number :  
    Days         : clinic days sorted with Monday as first day of week and Friday as last day of the week



Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.

Answer the customer in a friendly tone. Express concern that they are having a problem.
"""

In [13]:
list_user_messages = ['I may have broken my finger',
                      'Can I get my eyes checked',
                      'My kid needs his school physical',
                      'I need to schedule my annual Pap smear']

list_user_messages = ['I may have broken my finger']

# list_user_messages = ['Where can I get my teeth cleaned?' ]

# user_message = f"""
# Can I schedule an appointment with an ophthalmologist ? If I can what is their availability"""
cost_for_all_queries = 0
for message in list_user_messages[:2]:
    print(f'\n\nPatient asks: \033[91m\033[1m{message}\033[0m')

    user_message= 'f"""' + message + '"""'
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 

    # response = get_completion_from_messages(messages)
    # print(response)
                                 
    model="gpt-3.5-turbo"
    temperature=0 
    max_tokens=1_500
    
    response_details = get_completion_from_messages_FULL(messages,
                                                         model = model,
                                                         temperature = temperature, 
                                                         max_tokens = max_tokens)
    print(response_details)
    response = response_details.choices[0].message["content"]
    # print(response)
    try:
        final_response = response.split(delimiter)[-1].strip()
    except Exception as e:
        final_response = "Sorry, I'm having trouble right now, please try asking another question."

    # print(final_response)
    cost = token_usage(response_details)
    cost_for_all_queries += cost
    print(f'Cost for this query:${cost}')
    print('\n'*2 ) 
print(f'Total Cost:${cost_for_all_queries}')

steps = response_details['choices'][0]['message']['content']

# for step in steps:
print(steps)  



Patient asks: I may have broken my finger
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1:#### The user is asking a question about a specific medical issue (possibly a broken finger).\n\nStep 2:#### The appropriate specialty for a broken finger would be Orthopedics.\n\nStep 3:#### Creating a list of doctors in the Orthopedics specialty.\n\nStep 4:#### Creating a list of clinics that have doctors in the Orthopedics specialty.\n\nStep 5:#### Creating a dictionary with Orthopedics as the primary key, clinics as the secondary keys, doctors as the tertiary keys, and the days of the week as the values.\n\nStep 6:#### Checking if the patient has any restrictions on the day of the week and removing doctors who do not work on that day.\n\nResponse to user:#### I'm sorry to hear that you may have broken your finger. Let me check for Orthopedics clinics and doctors who can help you with your condition.",
        "role": "assi

In [11]:
print(steps)

Step 1:#### The user is asking a question about a specific medical issue (a possible broken finger).

Step 2:#### The appropriate specialty for a possible broken finger would be Orthopedics.

Step 3:#### Creating a list of doctors in the Orthopedics specialty.

Step 4:#### Creating a list of clinics that have doctors in the Orthopedics specialty.

Step 5:#### Creating a dictionary of clinics and doctors in the Orthopedics specialty.

Step 6:#### Creating a dictionary of clinic, doctor, and clinic days in the Orthopedics specialty.

Step 7:#### Checking if the patient has any restrictions on the day of the week.

Response to user:#### I'm sorry to hear that you may have broken your finger. Let me check for clinics and doctors specializing in Orthopedics who can help you.


In [16]:
df

,Physician,Specialty,Clinic,Location,Phone,Clinic_Days,Rating,Education,Years_in_Practice
0,Physician_1,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7911,"['Monday', 'Thursday', 'Tuesday']",4.4,{'Med_school': 'University of Wisconsin'},14
1,Physician_2,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-1098,"['Wednesday', 'Tuesday', 'Thursday']",3.1,{'Med_school': 'University of Miami'},25
2,Physician_3,Cardiology,Clinic_C,"123 Pine St, Anytown",111-111-8963,"['Wednesday', 'Tuesday', 'Thursday']",4.3,{'Med_school': 'University of Wisconsin'},23
3,Physician_4,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-6200,"['Wednesday', 'Tuesday', 'Monday']",4.9,{'Med_school': 'University of Wisconsin'},12
4,Physician_5,Pediatrics,Clinic_A,"123 Pine St, Anytown",111-111-3890,"['Thursday', 'Tuesday', 'Monday']",4.6,{'Med_school': 'UCDavis'},21
5,Physician_6,Internal Medicine,Clinic_C,"123 Pine St, Anytown",111-111-9954,"['Friday', 'Wednesday', 'Monday']",3.0,{'Med_school': 'University of Michigan'},22
6,Physician_7,OBGYN,Clinic_A,"123 Pine St, Anytown",111-111-5842,"['Monday', 'Thursday', 'Wednesday']",3.0,{'Med_school': 'University of Miami'},9
7,Physician_8,OBGYN,Clinic_B,"123 Pine St, Anytown",111-111-5879,"['Wednesday', 'Thursday', 'Tuesday']",4.8,{'Med_school': 'UCDavis'},21
8,Physician_9,Oncology,Clinic_C,"123 Pine St, Anytown",111-111-3164,"['Tuesday', 'Wednesday', 'Thursday']",3.2,{'Med_school': 'University of Miami'},12
9,Physician_10,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7448,"['Thursday', 'Monday', 'Wednesday']",3.2,{'Med_school': 'University of Michigan'},14


# Broken finger  version 1
data_list

{clinics_dict}


In [43]:
# data_list

In [57]:
delimiter = "####"
system_message = f"""
Acting as a customer service representative, respond to the patient \
The patient's query will be delimited with \
{delimiter} characters.

The health system has Doctors working in Specialties, working at particular clinics : 
{data_list}

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} Decide what specialty would be appropriate 

Step 3:{delimiter} Make a list of only doctors in that appropriate specialty

Step 4:{delimiter} Make a dictionary of those  doctors and their clinic days


Step 5:{delimiter} Select only those any  from  the dictionary 
that are  in that appropriate specialty





Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 3 reasoning>
Step 5:{delimiter} <step 3 reasoning>




If there are apprpriate clinics for the patient h
then list each of the physicians in the following format:

    Physician    :
    Specialty    :
    Clinic        :
    Phone Number :  
    Days         : clinic days sorted with Monday as first day of week and Friday as last day of the week



    Physician    :
    Specialty    :
    Clinic        :
    Phone Number :  
    Days         : clinic days sorted with Monday as first day of week and Friday as last day of the week



Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.

Answer the customer in a friendly tone. Express concern that they are having a problem.
"""

In [58]:
list_user_messages = ['I may have broken my finger',
                      'Can I get my eyes checked',
                      'My kid needs his school physical',
                      'I need to schedule my annual Pap smear']

list_user_messages = ['I may have broken my finger']

# list_user_messages = ['Where can I get my teeth cleaned?' ]

# user_message = f"""
# Can I schedule an appointment with an ophthalmologist ? If I can what is their availability"""
cost_for_all_queries = 0
for message in list_user_messages[:2]:
    print(f'\n\nPatient asks: \033[91m\033[1m{message}\033[0m')

    user_message= 'f"""' + message + '"""'
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 

    # response = get_completion_from_messages(messages)
    # print(response)
                                 
    model="gpt-3.5-turbo"
    temperature=0 
    max_tokens=1_500
    
    response_details = get_completion_from_messages_FULL(messages,
                                                         model = model,
                                                         temperature = temperature, 
                                                         max_tokens = max_tokens)
    print(response_details)
    response = response_details.choices[0].message["content"]
    # print(response)
    try:
        final_response = response.split(delimiter)[-1].strip()
    except Exception as e:
        final_response = "Sorry, I'm having trouble right now, please try asking another question."

    # print(final_response)
    cost = token_usage(response_details)
    cost_for_all_queries += cost
    print(f'Cost for this query:${cost}')
    print('\n'*2 ) 
print(f'Total Cost:${cost_for_all_queries}')

steps = response_details['choices'][0]['message']['content']

# for step in steps:
print(steps)  



Patient asks: I may have broken my finger
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1: The user is asking a question about a specific medical issue.\n\nStep 2: The appropriate specialty for a possible broken finger would be Orthopedics.\n\nStep 3: Making a list of doctors in the Orthopedics specialty.\n\nStep 4: Making a dictionary of the doctors in Orthopedics and their clinic days.\n\nStep 5: Selecting only the doctors in Orthopedics from the dictionary.\n\n#### Here are the doctors specializing in Orthopedics who can help you with your possible broken finger:\n\nPhysician    : Physician_1\nSpecialty    : Orthopedics\nClinic        : Clinic_B\nPhone Number : 111-111-7911\nDays         : Monday, Tuesday, Thursday\n\nPhysician    : Physician_2\nSpecialty    : Orthopedics\nClinic        : Clinic_B\nPhone Number : 111-111-1098\nDays         : Tuesday, Wednesday, Thursday\n\nPhysician    : Physician_4\nSpecialty  

In [ ]:
Odd th, but not when proper ordering.3 the answer is okat even thou the step are 3,3,

In [ ]:

df = df.sort_values(['Clinic'], ascending=[True])
df = df.reset_index(drop=True)
df

,Physician,Specialty,Clinic,Location,Phone,Clinic_Days,Rating,Education,Years_in_Practice
0,Physician_5,Pediatrics,Clinic_A,"123 Pine St, Anytown",111-111-3890,"['Thursday', 'Tuesday', 'Monday']",4.6,{'Med_school': 'UCDavis'},21
1,Physician_7,OBGYN,Clinic_A,"123 Pine St, Anytown",111-111-5842,"['Monday', 'Thursday', 'Wednesday']",3.0,{'Med_school': 'University of Miami'},9
2,Physician_12,Orthopedics,Clinic_A,"123 Pine St, Anytown",111-111-5186,"['Thursday', 'Monday', 'Friday']",5.0,{'Med_school': 'UCDavis'},6
3,Physician_1,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7911,"['Monday', 'Thursday', 'Tuesday']",4.4,{'Med_school': 'University of Wisconsin'},14
4,Physician_2,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-1098,"['Wednesday', 'Tuesday', 'Thursday']",3.1,{'Med_school': 'University of Miami'},25
5,Physician_4,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-6200,"['Wednesday', 'Tuesday', 'Monday']",4.9,{'Med_school': 'University of Wisconsin'},12
6,Physician_8,OBGYN,Clinic_B,"123 Pine St, Anytown",111-111-5879,"['Wednesday', 'Thursday', 'Tuesday']",4.8,{'Med_school': 'UCDavis'},21
7,Physician_10,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7448,"['Thursday', 'Monday', 'Wednesday']",3.2,{'Med_school': 'University of Michigan'},14
8,Physician_3,Cardiology,Clinic_C,"123 Pine St, Anytown",111-111-8963,"['Wednesday', 'Tuesday', 'Thursday']",4.3,{'Med_school': 'University of Wisconsin'},23
9,Physician_6,Internal Medicine,Clinic_C,"123 Pine St, Anytown",111-111-9954,"['Friday', 'Wednesday', 'Monday']",3.0,{'Med_school': 'University of Michigan'},22


In [86]:
print(steps)

Step 1: The user is asking a question about a specific medical issue.

Step 2: Based on the user's query, the appropriate specialty would be Orthopedics.

Step 3: Creating a list of doctors in the Orthopedics specialty.

Step 4: Creating a list of clinics that have doctors in the Orthopedics specialty.

Step 5: Creating a dictionary of clinics and the doctors who work in the Orthopedics specialty.

Step 6: Creating a dictionary of clinics, doctors, and their clinic days.

Step 7: Checking if the patient has any restrictions on the day of the week.

#### Based on your query, I recommend you see a doctor specializing in Orthopedics. Here are the clinics and doctors available:

Clinic: Clinic_B
Doctor 1: Physician_1
    Phone Number: 111-111-7911
    Clinic Days: Monday, Tuesday, Thursday

Doctor 2: Physician_2
    Phone Number: 111-111-1098
    Clinic Days: Tuesday, Wednesday, Thursday

Doctor 3: Physician_4
    Phone Number: 111-111-6200
    Clinic Days: Monday, Tuesday, Wednesday

Doct

# Specialty and Day of the Week

In [70]:
def process_a_prompt(system_message,
                    message) :  
    print(f'\n\nPatient asks: \033[91m\033[1m{message}\033[0m')

    user_message= 'f"""' + message + '"""'
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 

    # response = get_completion_from_messages(messages)
    # print(response)
                                 
    model="gpt-3.5-turbo"
    temperature=0 
    max_tokens=1_500
    
    response_details = get_completion_from_messages_FULL(messages,
                                                         model = model,
                                                         temperature = temperature, 
                                                         max_tokens = max_tokens)
    # print(response)
    try:
        final_response = response.split(delimiter)[-1].strip()
    except Exception as e:
        final_response = "Sorry, I'm having trouble right now, please try asking another question."

    # print(final_response)
    cost = token_usage(response_details)
    # cost_for_all_queries += cost
    print(f'Cost for this query:${cost}')
    print('\n'*2 ) 
    
    return response_details

In [71]:
    message = 'I broke my finger . Is there someone that can see me Thursday?'
    response_details = process_a_prompt(system_message, message)  
    print(response_details)
    response = response_details.choices[0].message["content"]



Patient asks: I broke my finger . Is there someone that can see me Thursday?
Cost for this query:$0.005



{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1: The user is asking a question about a specific medical issue.\n\nStep 2: The appropriate specialty for a broken finger would be Orthopedics.\n\nStep 3: Creating a list of doctors in the Orthopedics specialty.\n\nStep 4: Creating a list of clinics that have doctors in the Orthopedics specialty.\n\nStep 5: Creating a dictionary of clinics and the doctors that work in the Orthopedics specialty.\n\nStep 6: Creating a dictionary of clinics, doctors, and their clinic days.\n\nStep 7: Checking if there are any restrictions on Thursday for the doctors in the Orthopedics specialty.\n\n#### Based on the information provided, I have found the following clinics and doctors that specialize in Orthopedics:\n\nClinic: Clinic_B\nDoctor 1: Physician_1\n    Phone Number: 111-111-

In [72]:
steps = response_details['choices'][0]['message']['content']
print(steps)  

Step 1: The user is asking a question about a specific medical issue.

Step 2: The appropriate specialty for a broken finger would be Orthopedics.

Step 3: Creating a list of doctors in the Orthopedics specialty.

Step 4: Creating a list of clinics that have doctors in the Orthopedics specialty.

Step 5: Creating a dictionary of clinics and the doctors that work in the Orthopedics specialty.

Step 6: Creating a dictionary of clinics, doctors, and their clinic days.

Step 7: Checking if there are any restrictions on Thursday for the doctors in the Orthopedics specialty.

#### Based on the information provided, I have found the following clinics and doctors that specialize in Orthopedics:

Clinic: Clinic_B
Doctor 1: Physician_1
    Phone Number: 111-111-7911
    Clinic Days: Monday, Tuesday, Thursday

Doctor 2: Physician_2
    Phone Number: 111-111-1098
    Clinic Days: Tuesday, Wednesday, Thursday

Doctor 3: Physician_4
    Phone Number: 111-111-6200
    Clinic Days: Monday, Tuesday, We

In [64]:
print(steps.split('Response to user: ')[1])

I'm sorry to hear about your broken finger. Let me check if there are any orthopedic clinics available on Thursday for you.

Clinic      : Clinic_B
Doctor # 1  : Physician_1
    Phone Number : 111-111-7911
    Days         : Monday, Tuesday, Thursday
Doctor # 2  : Physician_2
    Phone Number : 111-111-1098
    Days         : Tuesday, Wednesday, Thursday
Doctor # 3  : Physician_10
    Phone Number : 111-111-7448
    Days         : Monday, Wednesday, Thursday

Clinic      : Clinic_A
Doctor # 1  : Physician_12
    Phone Number : 111-111-5186
    Days         : Monday, Thursday, Friday

Please let me know if you would like to schedule an appointment with any of these doctors.


In [79]:
df
mask = (df['Specialty'] == 'Orthopedics') 
df[mask] 

,Physician,Specialty,Clinic,Location,Phone,Clinic_Days,Rating,Education,Years_in_Practice
0,Physician_1,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7911,"['Monday', 'Thursday', 'Tuesday']",4.4,{'Med_school': 'University of Wisconsin'},14
1,Physician_2,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-1098,"['Wednesday', 'Tuesday', 'Thursday']",3.1,{'Med_school': 'University of Miami'},25
3,Physician_4,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-6200,"['Wednesday', 'Tuesday', 'Monday']",4.9,{'Med_school': 'University of Wisconsin'},12
9,Physician_10,Orthopedics,Clinic_B,"123 Pine St, Anytown",111-111-7448,"['Thursday', 'Monday', 'Wednesday']",3.2,{'Med_school': 'University of Michigan'},14
11,Physician_12,Orthopedics,Clinic_A,"123 Pine St, Anytown",111-111-5186,"['Thursday', 'Monday', 'Friday']",5.0,{'Med_school': 'UCDavis'},6


In [80]:
# Define the element to search for
element = 'Thursday'
is_present = df['Clinic_Days'].apply(lambda x: element in x).any()

if is_present:
    print(f"The element '{element}' is present in the DataFrame column.")
else:
    print(f"The element '{element}' is not present in the DataFrame column.")

The element 'Thursday' is present in the DataFrame column.


In [76]:


# Create a sample DataFrame
data = {'Column1': [['apple', 'banana', 'orange'], ['cat', 'dog'], ['tree', 'flower', 'grass']]}
df_1 = pd.DataFrame(data)

# Define the element to search for
element = 'dog'

# Check if the element is present in the DataFrame column
is_present = df_1['Column1'].apply(lambda x: element in x).any()

if is_present:
    print(f"The element '{element}' is present in the DataFrame column.")
else:
    print(f"The element '{element}' is not present in the DataFrame column.")

The element 'dog' is present in the DataFrame column.


In [55]:
list_Specialties = list(set(df.Specialty))
print(f'We have the following specialties')
for num, name in enumerate(list_Specialties, start=1): 
    print("{0:3}: {1}".format(num, name))
 

We have the following specialties
  1: Internal Medicine
  2: Oncology
  3: Orthopedics
  4: OBGYN
  5: Pediatrics
  6: Cardiology


In [66]:
mask = df['Specialty'] == 'Pediatrics'
df[mask] 


,Physician,Specialty,Clinic,Location,Phone,Clinic_Days,Rating,Education,Years_in_Practice
4,Physician_5,Pediatrics,Clinic_A,"123 Pine St, Anytown",111-111-3890,"['Thursday', 'Tuesday', 'Monday']",4.6,{'Med_school': 'UCDavis'},21
10,Physician_11,Pediatrics,Clinic_C,"123 Pine St, Anytown",111-111-4558,"['Tuesday', 'Friday', 'Thursday']",4.8,{'Med_school': 'UCDavis'},16


In [81]:
    message = 'My kid needs a school physical next Friday '
    response_details = process_a_prompt(system_message, message)  
    print(response_details)
    response = response_details.choices[0].message["content"]



Patient asks: My kid needs a school physical next Friday 
Cost for this query:$0.004



{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1: The user is asking a question about a specific medical issue, which is a school physical for their kid.\n\nStep 2: The appropriate specialty for a school physical would be Pediatrics.\n\nStep 3: Creating a list of all doctors in the Pediatrics specialty.\n\nStep 4: Creating a list of all clinics that have doctors in the Pediatrics specialty.\n\nStep 5: Creating a dictionary of clinics and the doctors who work in the Pediatrics specialty.\n\nStep 6: Creating a dictionary of clinics, doctors, and the days they work in the clinic.\n\nStep 7: Checking if there are any restrictions on the day of the week for the patient. If there are, removing doctors who do not work on the specified day.\n\n#### Based on your request for a school physical for your kid, I have found the following clini

In [82]:
print(response)

Step 1: The user is asking a question about a specific medical issue, which is a school physical for their kid.

Step 2: The appropriate specialty for a school physical would be Pediatrics.

Step 3: Creating a list of all doctors in the Pediatrics specialty.

Step 4: Creating a list of all clinics that have doctors in the Pediatrics specialty.

Step 5: Creating a dictionary of clinics and the doctors who work in the Pediatrics specialty.

Step 6: Creating a dictionary of clinics, doctors, and the days they work in the clinic.

Step 7: Checking if there are any restrictions on the day of the week for the patient. If there are, removing doctors who do not work on the specified day.

#### Based on your request for a school physical for your kid, I have found the following clinics and doctors in the Pediatrics specialty:

Clinic: Clinic_A
Doctor 1: Physician_5
    Phone Number: 111-111-3890
    Clinic Days: Monday, Tuesday, Thursday

Clinic: Clinic_C
Doctor 1: Physician_11
    Phone Number

# Stopping  June 29, 2023  
Some Markdown text with <span style="color:red">some *red* text</span>.

<div class="alert alert-block alert-danger">problem is making up ophth 
    and there is only one pediatrc clinic 

</div>

Check the response before it is sent.

In [18]:
url_chatGPT_usage = 'https://platform.openai.com/account/usage'
webbrowser.open_new(url_chatGPT_usage)

True

In [160]:
user_message_1 = f"""
I  broke my finger """
print(f'\033[91m\033[1m{user_message_1}\033[0m') 

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 



response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
print(response)
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."



# print(final_response)
cost = token_usage(response_details)
# cost_for_all_queries += cost
print(f'Cost for this query:${cost}')    


I  broke my finger 
Step 1:#### The user is asking a question about a specific medical issue.
Step 2:#### Orthopedics would be the appropriate specialty for a broken finger.

The following clinics specialize in Orthopedics:
Clinic      : Orthopedics Clinic A
Location    : 123 Main Street, City A
Phone       : 111-111-1111
Doctors     : Doctor 1 works on Monday, Wednesday, Friday
              Doctor 2 works on Tuesday, Thursday
              Doctor 3 works on Monday, Thursday

Clinic      : Orthopedics Clinic B
Location    : 456 Elm Street, City B
Phone       : 222-222-2222
Doctors     : Doctor 4 works on Tuesday, Wednesday, Friday
              Doctor 5 works on Monday, Thursday
              Doctor 6 works on Tuesday, Thursday

The Orthopedics clinics are open on weekdays from 9am to 5pm. 
Please call the clinic phone number to make an appointment.
Cost for this query:$0.002


In [161]:
url_chatGPT_usage = 'https://platform.openai.com/account/usage'
webbrowser.open_new(url_chatGPT_usage)


True

## Get Keys

In [44]:

def get_specialty_by_name(data):
    
    return data.keys()

def get_clinic_by_name(data):
    
    return data.keys()

def get_doctors_by_clinic(clinic):
    return clinic.keys()

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [26]:
delimiter = "####"
system_message = f"""
Acting as a customer service agent,
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific procedure.  

Step 2:{delimiter} If the user is asking about \
specific procedure, identify whether \
the procedure are in the following list.
All available procedures: 
{json_data}


Step 3:{delimiter} If the message contains a procedure \
in the list above, list any assumptions that the \
user is making in their \
message e.g. They are wanting to schedule a procedure.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
procedure information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference procedure in the list of \
4 available procedures, as these are the only 4 \
procedure that are available to the patient. \
Answer the customer in a friendly tone. 

If the customer is asking for a procedure that is on the list, 
then list each of the following on a separate line in the following format:
Procedure   : list procedure
Location    : Location
Address     : Address
Availability: Availability

If there are more than one location for a given procedure, be sure to list ALL locations 

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

# Eye Exam

In [27]:
user_message = f"""  Can I schedule eye exam?"""
print(f'\n\nPatient asks: {user_message}')
    
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

# response = get_completion_from_messages(messages)
# print(response)
response = get_completion_from_messages_FULL(messages)

response = response.choices[0].message["content"]
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)




Patient asks:   Can I schedule eye exam?
I'm sorry, but we do not offer eye exams at this time. Our available procedures are Orthopedics, Internal Medicine, OB/GYN, and Pediatrics. Is there anything else I can assist you with?


## Stepwise Processing (6 Steps)

In [28]:

delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} If the user is asking about \
specific medical issue, identify whether the issue could be managed by a\
 medical specialty in the following list.
All available locations: 
{json_data}

Step 3:{delimiter} If the message contains a medical condition that could be managed by a medical specialty \
in the list above, list any assumptions that the \
user is making in their \
message e.g. They are wanting to schedule an appointment with a particular clinic.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on the \
medical specialty information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference medical clinics in the list of \ 
available medical clinics, as these are the only  \
medical clinics that are available to the patient. \
Answer the customer in a friendly tone. Express concern that they are having a problem.

Step 6:{delimiter}: 
If the patient has a question that could be managed by a medical clinic that is on the list, 
then, on a separate line for each,  for each clinic in that specialty list the  clinic name, clinic location, and phone number, as follows
Clinic      :
Location    :
Phone       :
On a separate line, inform the patient which days the clinic is availble. Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.


Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 3 reasoning>
Step 6:{delimiter} <step 4 reasoning>

Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

## 5 Steps

In [3]:

delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} If the user is asking about \
specific medical issue, identify whether the issue could be managed by a\
 medical specialty in the following list.
All available locations: 
{json_data}

Step 3:{delimiter} If the message contains a medical condition that could be managed by a medical specialty \
in the list above, list any assumptions that the \
user is making in their \
message e.g. They are wanting to schedule an appointment with a particular clinic.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on the \
medical specialty information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference medical clinics in the list of \ 
available medical clinics, as these are the only  \
medical clinics that are available to the patient. \
Answer the customer in a friendly tone. Express concern that they are having a problem.



Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 3 reasoning>

If the patient has a question that could not be managed by a medical clinic that is on the list, respond in a polite
manner.



If the patient has a question that could be managed by a medical clinic that is on the list, 
then, on a separate line for each,  for each clinic in that specialty list the  clinic name, clinic location, and phone number, as follows
Clinic      :
Location    :
Phone       :
On a separate line, inform the patient which days the clinic is availble. Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.


Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

user_message = f"""  I may have broken my finger"""
print(f'\n\nPatient asks: {user_message}')

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

# response = get_completion_from_messages(messages)
# print(response)
response = get_completion_from_messages_FULL(messages)

response = response.choices[0].message["content"]
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)
# final_response = response_dict['choices'][0]['message']['content'].split('####')[-1][1:]
# print(final_response)

In [35]:
final_response

'If you would like to schedule your appointment at an OB/GYN clinic, here is the information for the available clinics:\n\nClinic      : OB/GYN Clinic A\nLocation    : 567 Pine Street, City E\nPhone       : 555-555-5555\nThe clinic is open on Monday, Tuesday, Wednesday, Thursday, and Friday.\n\nClinic      : OB/GYN Clinic B\nLocation    : 789 Oak Street, City F\nPhone       : 666-666-6666\nThe clinic is open on Monday, Tuesday, Wednesday, Thursday, and Friday.\n\nPlease call the phone number of the clinic you would like to schedule your appointment with.'

# Check if there is an appropriate clinic

In [51]:
question = """
If the patient has a question that could be managed by a medical clinic that is on the list, 
then, on a separate line for each,  
for each clinic in that specialty list the  clinic name, clinic location, and phone number, as follows
Clinic      :
Location    :
Doctors working at the Clinic and their office days:
"""

# question = """
# If the patient has a question that could be managed by a medical clinic that is on the list, 
# then, create a dictionary with the 
# specialty name as primary key,
# doctors name as secondary key and
# doctors availability as values
# """

In [52]:
delimiter = "####"
system_message = f"""

You are a customer service representative that 
informs patients about what days specific clinics are open.
The patient will describe a medical problem and you should determine if we have a clinic that
would be appropriate where the patient could be seen.

If the patient has a question that could not be managed by a medical clinic that is on the list, 
respond in a polite manner.

All available locations: 
{json_data}


{question}

Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

user_message = f"""  I may have broken my finger"""
print(f'\n\nPatient asks: {user_message}')

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages_FULL(messages)

response



Patient asks:   I may have broken my finger


<OpenAIObject chat.completion id=chatcmpl-7Pe3xGEI9vvEaNFyRs6bFaTmhrQfB at 0x7fb265335900> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I'm sorry to hear that. We have Orthopedics clinics that specialize in bone and joint injuries. Would you like me to check if we have a clinic near you?",
        "role": "assistant"
      }
    }
  ],
  "created": 1686346557,
  "id": "chatcmpl-7Pe3xGEI9vvEaNFyRs6bFaTmhrQfB",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 35,
    "prompt_tokens": 375,
    "total_tokens": 410
  }
}

In [58]:
json_data
data
for k,v in data.items():
    print(k)
get_specialty_by_name(data)

Orthopedics


dict_keys(['Orthopedics'])

dict_keys(['Dental Check-up', 'Eye Exam', 'Vaccination', 'Physical Therapy'])

In [15]:
def get_specialty_by_name(data):
    
    return data.keys()

def get_clinic_by_name(data):
    
    return data.keys()

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

## Electronics


In [110]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""
user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]


In [113]:
print('Internal Medicine Clinic A',
 'Internal Medicine Clinic B',
 'OB/GYN Clinic A',
 'OB/GYN Clinic B',
 'Orthopedics Clinic A',
 'Orthopedics Clinic B',
 'Pediatrics Clinic A',
 'Pediatrics Clinic B')

Internal Medicine Clinic A Internal Medicine Clinic B OB/GYN Clinic A OB/GYN Clinic B Orthopedics Clinic A Orthopedics Clinic B Pediatrics Clinic A Pediatrics Clinic B


In [116]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'Specialty': <one of Orthopedics, Internal Medicine,OB/GYN,
    Pediatrics>,
OR
    'Clinics': <a list of Clinics that must \
    be found in the allowed Clinics below>

Where the Specialty and Clinics must be found in \
the customer service query.
If a Clinics is mentioned, it must be associated with \
the correct category in the allowed Clinics list below.
If no Clinics or Specialty are found, output an \
empty list.

Allowed Clinics: 

Internal Medicine Clinics:
Internal Medicine Clinic A 
Internal Medicine Clinic B 

OB/GYN Clinics:
OB/GYN Clinic A 
OB/GYN Clinic B 

Orthopedics Clinics:
Orthopedics Clinic A 
Orthopedics Clinic B 

Pediatrics Clinics:
Pediatrics Clinic A 
Pediatrics Clinic B


Only output the list of Specialties, with nothing else.
"""
user_message_1 = f"""
 tell me about the Specialties that you have """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[{'Specialty': 'Orthopedics'}, {'Specialty': 'Internal Medicine'}, {'Specialty': 'OB/GYN'}, {'Specialty': 'Pediatrics'}]


In [ ]:
Clinic_Names_Locations

## Clinics - Names and Location

In [121]:
Clinics = {
    "Pediatrics Clinic A": {
        "name": "Pediatrics Clinic A",
        "location": "Main Street",
        "phone": "222-222-2222",
        "doctors":  {"Doctor A" : ["Monday, Wednesday, Friday"],
                   "Doctor B" : ["Monday,Tuesday, Wednesday"],
                    }

    },

    "Pediatrics Clinic B": {
        "name": "Pediatrics Clinic B",
        "location": "Pine Street",
        "phone": "222-333-2222",
        "doctors":  {"Doctor A" : ["Monday, Wednesday, Friday"]}

    }
}
type(Clinics)

dict

## Do we have a clinic - broke finger

In [132]:
delimiter = "####"
system_message = f"""
Act as if you are a customer service rep. \
The customer's query will be delimited with \
{delimiter} characters.

The following lists are  Clinics: 
{Clinics}


Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 1:{delimiter} Decide what specialty would be appropriate.  


Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>


If we do not have a medical clinic that is relevenat to the patient's problme
, respond in a polite manner and advise them to search the web to find one.



If the patient has a question that could be managed by a medical clinic
that is on the list, 
then list each of the relevant clinics.
Clinic      :
Location    :
Phone       :
On a separate line, inform the patient which days the clinic is availble. Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.


Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.

Answer the customer in a friendly tone. Express concern that they are having a problem.
If we have an appropriate clinic,on separate lines, list the following information in the following format:
Location:
Phone :
"""
user_message_1 = f"""
I  broke my finger """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 



response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
print(response)
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."



Step 1:#### The user is asking about a specific medical issue.
Step 2:#### Orthopedics would be the appropriate specialty for a broken finger.

We do not have an orthopedic clinic on our list. I suggest you search the web to find a clinic that can help you with your broken finger.


In [ ]:
, on a separate line for each,  for each clinic in that specialty list the  clinic name, clinic location, and phone number, as follows


### product information

In [14]:
# product information
products = {
    "TechPro Ultrabook": {
        "name": "TechPro Ultrabook",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 processor"],
        "description": "A sleek and lightweight ultrabook for everyday use.",
        "price": 799.99
    },

    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "A powerful desktop computer for work and play.",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "A compact and affordable Chromebook for everyday tasks.",
        "price": 249.99
    },

    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "Experience true wireless freedom with these comfortable earbuds.",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "A stunning 4K TV with vibrant colors and smart features.",
        "price": 599.99
    },
 
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "Experience the future of television with this stunning 8K TV.",
        "price": 2999.99
    },


 
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "Create instant memories with this fun and portable instant camera.",
        "price": 69.99
    }
}

In [64]:
delimiter = "####"
system_message = f"""

You are a customer service representative that 
informs patients about what days specific clinics are open.
The patient will describe a medical problem and you should determine if we have a clinic that
would be appropriate where the patient could be seen.

If the patient has a question that could not be managed by a medical clinic that is on the list, 
respond in a polite manner.

All available locations: 
{data}

"""

user_message = f"""
 tell me about the medical clinics that you have"""

print(f'\n\nPatient asks: {user_message}')
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
clinics_response = get_completion_from_messages(data)
print(clinics_response)






Patient asks: 
 tell me about the medical clinics that you have


InvalidRequestError: {'Orthopedics': {'Orthopedics Clinic A': {'Doctor 1': ['123 Main Street, City A', '111-111-1111', 'Monday, Wednesday, Friday'], 'Doctor 2': ['123 Main Street, City A', '111-111-1111', 'Tuesday, Thursday'], 'Doctor 3': ['123 Main Street, City A', '111-111-1111', 'Monday, Thursday']}, 'Orthopedics Clinic B': {'Doctor 4': ['456 Elm Street, City B', '222-222-2222', 'Tuesday, Wednesday, Friday'], 'Doctor 5': ['456 Elm Street, City B', '222-222-2222', 'Monday, Thursday'], 'Doctor 6': ['456 Elm Street, City B', '222-222-2222', 'Tuesday, Thursday']}}} is not of type 'array' - 'messages'

In [ ]:
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)


In [44]:
# response = get_completion_from_messages(messages)
# print(response)
response = get_completion_from_messages_FULL(messages)

response = response.choices[0].message["content"]
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)
# final_response = response_dict['choices'][0]['message']['content'].split('####')[-1][1:]
# print(final_response)



Patient asks:   I may have broken my finger
I'm sorry to hear that. We have two Orthopedics clinics that may be able to assist you. They are Orthopedics Clinic A and Orthopedics Clinic B. Which one would you like more information on?


In [45]:
response

"I'm sorry to hear that. We have two Orthopedics clinics that may be able to assist you. They are Orthopedics Clinic A and Orthopedics Clinic B. Which one would you like more information on?"

In [ ]:
Phone       :
On a separate line, inform the patient which days the clinic is availble. Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.


In [ ]:
system_message = f"""
You are a customer service rep who informs patients about what days a clinic is open
assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

## Questionning the Responses  
- is clinic availability correct?


In [ ]:
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

In [ ]:

delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific medical issue.  

Step 2:{delimiter} If the user is asking about \
specific medical issue, identify whether the issue could be managed by a\
 medical specialty in the following list.
All available locations: 
{json_data}

Step 3:{delimiter} If the message contains a medical condition that could be managed by a medical specialty \
in the list above, list any assumptions that the \
user is making in their \
message e.g. They are wanting to schedule an appointment with a particular clinic.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on the \
medical specialty information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference medical clinics in the list of \ 
available medical clinics, as these are the only  \
medical clinics that are available to the patient. \
Answer the customer in a friendly tone. Express concern that they are having a problem.



Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 3 reasoning>

If the patient has a question that could not be managed by a medical clinic that is on the list, respond in a polite
manner.



If the patient has a question that could be managed by a medical clinic that is on the list, 
then, on a separate line for each,  for each clinic in that specialty list the  clinic name, clinic location, and phone number, as follows
Clinic      :
Location    :
Phone       :
On a separate line, inform the patient which days the clinic is availble. Phrase it as "The name of clinic is open on ..."
On a separate line, ask the patient to call the clinic phone number to make an appointment.


Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

user_message = f"""  I may have broken my finger"""
print(f'\n\nPatient asks: {user_message}')

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

# response = get_completion_from_messages(messages)
# print(response)
response = get_completion_from_messages_FULL(messages)

response = response.choices[0].message["content"]
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)
# final_response = response_dict['choices'][0]['message']['content'].split('####')[-1][1:]
# print(final_response)

In [34]:
type(json_data)
'Ophthalmology' in json_data

False

In [31]:
list_user_messages = ['I may have broken my finger',
                      'Can I get my eyes checked',
                     'My kid needs his school physical',
                     'I need to schedule my annual Pap smear']

# list_user_messages = ['Where can I get my teeth cleaned?' ]

# user_message = f"""
# Can I schedule an appointment with an ophthalmologist ? If I can what is their availability"""
cost_for_all_queries = 0
for message in list_user_messages[1:]:
    print(f'\n\nPatient asks: \033[91m\033[1m{message}\033[0m')

    user_message= 'f"""' + message + '"""'
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 

    # response = get_completion_from_messages(messages)
    # print(response)
    response_details = get_completion_from_messages_FULL(messages)
    response = response_details.choices[0].message["content"]
    # print(response)
    try:
        final_response = response.split(delimiter)[-1].strip()
    except Exception as e:
        final_response = "Sorry, I'm having trouble right now, please try asking another question."

    print(final_response)
    cost = token_usage(response_details)
    cost_for_all_queries += cost
    print(f'Cost for this query:${cost}')
print(f'Total Cost:${cost_for_all_queries}')



Patient asks: Can I get my eyes checked
Clinic      : Ophthalmology Clinic A
Location    : 123 Main Street, City A
Phone       : 111-111-1111
The Ophthalmology Clinic A is open on Monday, Wednesday, and Friday.

Please call the clinic phone number to make an appointment.
Cost for this query:$0.003


Patient asks: My kid needs his school physical
If the patient confirms that they are looking for a Pediatrics clinic, we can provide the following information:

Clinic      : Pediatrics Clinic A
Location    : 123 Elm Street, City G
Phone       : 777-777-7777
The clinic is open on Monday, Wednesday, and Friday.

Clinic      : Pediatrics Clinic B
Location    : 987 Pine Street, City H
Phone       : 888-888-8888
The clinic is open on Monday, Wednesday, and Friday.

Please call the clinic phone number to schedule an appointment.
Cost for this query:$0.003


Patient asks: I need to schedule my annual Pap smear
If you would like to schedule your appointment at an OB/GYN clinic, here is the infor

In [22]:
print(f'\033[91m\033[1m{message} \033[0m')


I may have broken my finger 


In [48]:
system_message

'\nFollow these steps to answer the customer queries.\nThe customer query will be delimited with four hashtags,i.e. ####. \n\nStep 1:#### First decide whether the user is asking a question about a specific medical issue.  \n\nStep 2:#### If the user is asking about specific medical issue, identify whether the issue could be managed by a medical specialty in the following list.\nAll available locations: \n{"Orthopedics": {"Orthopedics Clinic A": ["123 Main Street, City A", "111-111-1111", "Doctor 1", "Monday, Wednesday, Friday"], "Orthopedics Clinic B": ["456 Elm Street, City B", "222-222-2222", "Doctor 4", "Tuesday, Wednesday, Friday"]}, "Internal Medicine": {"Internal Medicine Clinic A": ["789 Oak Street, City C", "333-333-3333", "Doctor 7", "Monday, Wednesday, Friday"], "Internal Medicine Clinic B": ["321 Maple Street, City D", "444-444-4444", "Doctor 10", "Monday, Wednesday, Friday"]}, "OB/GYN": {"OB/GYN Clinic A": ["567 Pine Street, City E", "555-555-5555", "Doctor 13", "Monday, Tu

In [36]:
print(response)

#### It sounds like you may have a medical issue. Would you like me to help you find a medical clinic that can assist you? ####


In [32]:
response = response.choices[0].message["content"]
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)


AttributeError: 'str' object has no attribute 'choices'

In [63]:
user_message = f"""  I may have broken my finger"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)


Step 1:#### The user is asking about a specific medical issue.
Step 2:#### The issue could be managed by Orthopedics.
Step 3:#### The user is assuming that they need to schedule an appointment with a specific clinic.
Step 4:#### The assumption is not true until the user specifies which clinic they want to visit.
Step 5:#### Hello! I'm sorry to hear that you may have broken your finger. To better assist you, could you please specify which clinic you would like to visit? We have two Orthopedics clinics available: Orthopedics Clinic A and Orthopedics Clinic B.
Step 6:#### Once you specify which clinic you would like to visit, I can provide you with the clinic name, location, phone number, and availability.


In [57]:
response

"Step 1:#### The user is asking about a specific medical issue.\n\nStep 2:#### The issue could be managed by Orthopedics.\n\nStep 3:#### No assumptions made.\n\nStep 4:#### No incorrect assumptions made.\n\nStep 5:#### I'm sorry to hear that you may have broken your finger. Our Orthopedics clinics can help you with that. \n\nStep 6:#### \nClinic      : Orthopedics Clinic A\nLocation    : 123 Main Street, City A\nPhone       : 111-111-1111\nThe Orthopedics Clinic A is open on Monday, Wednesday, and Friday.\nPlease call the clinic phone number to make an appointment.\n\nClinic      : Orthopedics Clinic B\nLocation    : 456 Elm Street, City B\nPhone       : 222-222-2222\nThe Orthopedics Clinic B is open on Tuesday, Wednesday, and Friday.\nPlease call the clinic phone number to make an appointment.\n\nResponse to user:#### Please call one of the Orthopedics clinics listed above to schedule an appointment."

## Check the validity/relevance of output

In [8]:
check_response = openai.Moderation.create(
    input=response
)
moderation_output = check_response["results"][0]
print(moderation_output)

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 1.9721957e-08,
    "hate/threatening": 3.3151053e-09,
    "self-harm": 4.7006935e-08,
    "sexual": 2.1202966e-06,
    "sexual/minors": 1.9148263e-09,
    "violence": 3.5786761e-06,
    "violence/graphic": 3.394886e-06
  },
  "flagged": false
}


In [13]:
final_response_to_customer =  final_response
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
# customer_message = f"""
# tell me about the smartx pro phone and \
# the fotosnap camera, the dslr one. \
# Also tell me about your tvs"""

customer_message = f"""  I may have broken my finger"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

N


# End - and Extras

In [19]:
# # clinics = create_dictionary_from_csv(csv_file_path)
# clinics = CSV_to_DICT(csv_file_path)


# print(f'data is of {type(clinics)}')
# json_clinics = json.dumps(clinics)
# print(f'data is of {type(json_clinics)}')
# # json_data
# result = pd.concat([df.head(3), df.tail(3)])


# # print(result)

# # pp_json(json_clinics)

# clinics_dict = json_to_dict(json_clinics)
# print(f'json_clinics is of type ; {type(json_clinics)}')
# print(f'clinics_dict is of type ; {type(clinics_dict)}')
# for k,v in clinics_dict.items():
#     print(k, v)
# result

In [7]:
# data = {
#     'Physician_1': {'Orthopedics': ['Clinic_B', '111-111-7911', "['Monday', 'Thursday', 'Tuesday']", '4.4', "{'Med_school': 'University of Wisconsin'}", '14']},
#     'Physician_2': {'Orthopedics': ['Clinic_B', '111-111-1098', "['Wednesday', 'Tuesday', 'Thursday']", '3.1', "{'Med_school': 'University of Miami'}", '25']},
#     'Physician_3': {'Cardiology': ['Clinic_C', '111-111-8963', "['Wednesday', 'Tuesday', 'Thursday']", '4.3', "{'Med_school': 'University of Wisconsin'}", '23']},
#     'Physician_4': {'Orthopedics': ['Clinic_B', '111-111-6200', "['Wednesday', 'Tuesday', 'Monday']", '4.9', "{'Med_school': 'University of Wisconsin'}", '12']}
# }

# clinic_dict = {}
# for physician, info in data.items():
#     clinic_name = info[list(info.keys())[0]][0]  # Get the clinic name from the first specialty
#     if clinic_name not in clinic_dict:
#         clinic_dict[clinic_name] = []
#     clinic_dict[clinic_name].append({physician: info})

# print(clinic_dict)
type(data_list)

list

In [ ]:
def CSV_to_DICT(file_name): 
    # csv_file = "4_specialties_multiple_locations.csv"  # Replace with the path to your CSV file
    assert '.csv' in file_name, f'{csv_file} is NOT a CSV file'
    data = {}

    # Read the CSV file
    with open(file_name, mode="r") as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the header row

        for row in reader:
            primary_key = row[0]  # Primary key from the first column
            secondary_key = row[1]  # Secondary key from the second column
            tertiary_key = row[2]  # Secondary key from the second column
            values = row[3:]  # Values from the remaining columns

            if primary_key in data:
                # Primary key already exists, make the secondary key a nested dictionary
                if secondary_key not in data[primary_key]:
                    data[primary_key][secondary_key] = values
            else:
                # Primary key doesn't exist, create a new entry
                if tertiary_key in data:
                    data[primary_key][secondary_key][tertiary_key] = values
                else:
                    data[primary_key] = {secondary_key: values}

    # Print the resulting dictionary
    # print(data)
    return data


In [ ]:
data = {
    "Dental Check-up": {
        "Location": "Dental Clinic A",
        "Address": "123 Main Street, City A",
        "Availability": ["Monday", "Wednesday", "Friday"]
    },
    "Eye Exam": {
        "Location": "Eye Clinic B",
        "Address": "456 Elm Street, City B",
        "Availability": ["Tuesday", "Thursday"]
    },
    "Vaccination": {
        "Location": "Health Center C",
        "Address": "789 Oak Street, City C",
        "Availability": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    },
    "Physical Therapy": {
        "Location": "Therapy Center D",
        "Address": "321 Maple Street, City D",
        "Availability": ["Monday", "Tuesday", "Thursday", "Friday"]
    }
}
get_specialty_by_name(data)